In [29]:
! pip install beautifulsoup4

     |████████████████████████████████| 102kB 625kB/s ta 0:00:01


In [89]:
from bs4 import BeautifulSoup
import urllib.request as urllib2
import pandas as pd

Fetches data from wiki page

In [90]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
soup = BeautifulSoup(urllib2.urlopen(url).read())

Parses table header and cells from html data

In [91]:
table = soup.find(lambda tag: tag.name=='table') 
headers = table.findAll(lambda tag: tag.name=='th')
cells = table.findAll(lambda tag: tag.name=='td')

Cleans html tags and new line character

In [92]:
def cleantags(htmlTag):
    s = str(htmlTag)
    tag = False
    out = ""

    for c in s:
            if c == '<':
                tag = True
            elif c == '>':
                tag = False
            elif not tag:
                out = out + c
    out = out.replace('\n', '')
    return out

Creates empty df

In [94]:
dfHeaders = [cleantags(headers[0]), cleantags(headers[1]), cleantags(headers[2])]
df = pd.DataFrame(columns=dfHeaders)

Adds rows to df

In [95]:
rowDf = pd.DataFrame()
row = ['','','']
for i in range(int(len(cells) / 3)):
    for j in range(3):
        row[j] = cleantags(cells[3*i+j])
    rowDf = pd.DataFrame([row], columns=dfHeaders)
    df = df.append(rowDf, ignore_index=True)

Deletes if Borough is not assigned

In [96]:
df = df.loc[df['Borough'] != 'Not assigned']
df = df.reset_index()
df = df.drop('index', 1)

Below two cells merge neighbourhoods if postal codes are same

In [97]:
postCode = ''
for index, row in df.iterrows():
    if row['Postcode'] != postCode:
        postCode = row['Postcode']
    else:
        row['Neighbourhood'] = df.iloc[index - 1]['Neighbourhood'] + ', ' + row['Neighbourhood']
        df.iloc[index -1]['Neighbourhood'] = 'merged'

In [98]:
df = df.loc[df['Neighbourhood'] != 'merged']
df = df.reset_index()
df = df.drop('index', 1)

In [99]:
df[df['Neighbourhood'] == 'Not assigned']

,Postcode,Borough,Neighbourhood
4,M7A,Queen's Park,Not assigned


If neighbourhood is not assigned, assigns borough

In [100]:
df.loc[df['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = df[df['Neighbourhood'] == 'Not assigned']['Borough']
df[df['Postcode'] == 'M7A']

,Postcode,Borough,Neighbourhood
4,M7A,Queen's Park,Queen's Park


Saving the data frame and showing:

In [101]:
df.to_csv(r'toronto_Neighbourhoods.csv')
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [102]:
df.shape

(103, 3)